# Axis of interest

In [1]:
import os
import json
import lmstudio as lms
import sys
import re
from langchain_openai import ChatOpenAI
from pathlib import Path

repo_root = Path(__file__).resolve().parents[1] if "__file__" in globals() else Path("/Users/jpconde/Facultad/tesis")
if str(repo_root) not in sys.path:
    sys.path.append(str(repo_root))

In [2]:
from axis_of_interest.schemas import list_of_aoi
from axis_of_interest.prompts import prompt_generate_plot_schema
from axis_of_interest.utils import render_aoi_md, render_plot_schema_md

### Crear AoI en python

Para eso usamos pydantic, creamos una función que dados los AoI que nos pasó el Dr.Gervás creamos las instancias en tiempo real


In [3]:
for aoi in list_of_aoi:
    print(render_aoi_md(aoi))
    print("\n")


AXISofINTEREST =  CALL TO ACTION REWARD
PROTAGONIST =  called
ROLES =  called caller rewarded
────────────────────────────────────────────────────────────────
PLOT-SPAN-NAME =  Call

  CallToAction  characters(called=Hero,caller=Sender)

────────────────────────────────────────────────────────────────
PLOT-SPAN-NAME =  Reward

  Reward  characters(rewarded=Hero)

────────────────────────────────────────────────────────────────



AXISofINTEREST =  CONFLICT
PROTAGONIST =  attacker
ROLES =  attacker defender winner loser
────────────────────────────────────────────────────────────────
PLOT-SPAN-NAME =  Struggle

  Struggle  characters(attacker=X,defender=Y)

────────────────────────────────────────────────────────────────
PLOT-SPAN-NAME =  Victory

  Victory  characters(winner=X,loser=Y)

────────────────────────────────────────────────────────────────



AXISofINTEREST =  DONOR
PROTAGONIST =  tested
ROLES =  tested tester user gift
───────────────────────────────────────────────────────

### Crear Plot Schema

Le pegamos a una api de un LLM, con algunos axes of interest en el prompt y le pedimos que nos genere un Plot Schema

### Crear Cuento Completo

Dado un Plot Schema usamos un LLM para generar un cuento

### Crear AoI dado un cuento

Acá lo que hacemos es el paso inverso. Queremos mandar un cuento y ver si el LLM es capaz de detectar los AoI. Hicimos dos experimentos:
- Con el cuento que creamos recien y ver si nos devuelve los AoI que usamos
- Con un cuento cualquiera, este cuento nosotros lo analizamos nosotros y detectamos algunos AoI